In [1]:
import pandas as pd
import numpy as np
import time
from scipy.stats import kurtosis
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
def mean_absolute_percentage_error(actual, prediction):
    actual = pd.Series(actual)
    prediction = pd.Series(prediction)
    return 100 * np.mean(np.abs((actual - prediction))/actual)

data = pd.read_csv('NVDA_bar_sample_1min.csv', index_col=0, header=0)
display(data)
data = data.loc['2020-09-04 09:30:00':'2020-09-04 3:30:00'].reset_index(drop=False)
# display(data)
data['volume'] = data['volume'].astype('int32')
# del data["average"]
del data["barcount"]

def calcArima(data,column,len_test=10,diff = 1 ):
    try:del data['date']
    except: pass
    len_data = len(data)
    len_train = len_data-len_test-1
    test_starters = data.iloc[[len_train]]
#     display(data)
    test_data_orig = data.tail(len_test)
    if diff == 1:
        datad = data[column].diff()
        datad= datad.drop(data.index[0])
#         print(datad)
    test_data= datad.tail(len_test)
    train_data = datad.head(len_train)
#     display(train_data)
#     display(test_data)
#     return train_data,test_data,test_starters
    display(train_data.hist())
#     display(test_data.hist())
    start = test_starters[column].values[0]
#     print(start)
    train = train_data.values.tolist()
    test = test_data.values.tolist()
    test_len = len(test)
    # Initialize model
    model = pm.auto_arima(train, max_p=5, max_q=5, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)

    # Determine model parameters
    model.fit(train)
    order = model.get_params()['order']
    print('ARIMA order:', order, '\n')

    # Genereate predictions
    last_actual = start
    actuals = []
    prediction = []
    prediction_chain = []
    for i in range(len(test)):
#         print(test[i])
        model = pm.ARIMA(order=order)
        model.fit(train)
        print('working on', i+1, 'of', test_len, '-- ' + str(int(100 * (i + 1) / test_len)) + '% complete')
        prediction_list = model.predict()
        print(prediction_list)
        prediction_chain.append(prediction_list)
#         prediction.append(model.predict()[0])
#         print(prediction)
        train.append(test[i])
#         print(test[i])
        if diff == 1:
            prediction.append(last_actual+model.predict()[0])
            actuals.append(round(last_actual+test[i],2))
            last_actual = last_actual+test[i]
        else: actuals.append(test[i])
#         print(model.predict()[0],test[i],model.predict()[0]-test[i])
    # Generate error data
    mse = mean_squared_error(test, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(pd.Series(test), pd.Series(prediction))
    
    return test_data_orig,test_starters[column],prediction,actuals,prediction_chain, mse, rmse, mape



,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-09-04 04:00:00,512.00,512.98,512.00,512.98,21,512.047,3
2020-09-04 04:01:00,513.87,514.00,512.98,512.98,9,513.502,7
2020-09-04 04:02:00,512.98,512.98,512.98,512.98,0,512.980,0
2020-09-04 04:03:00,511.60,511.60,510.00,510.00,3,510.555,3
2020-09-04 04:04:00,505.21,505.22,505.20,505.22,12,505.206,7
...,...,...,...,...,...,...,...
2020-09-04 19:55:00,499.95,499.95,499.95,499.95,1,499.950,1
2020-09-04 19:56:00,499.95,499.95,499.95,499.95,0,499.950,0
2020-09-04 19:57:00,499.95,499.95,499.95,499.95,0,499.950,0


In [2]:
pcdf_list = list()
c = 0
for x in prediction_chain:
    pcdf = pd.DataFrame(x)
    pcdf.columns = ['pc']
    if c == 0: 
        s = test_starters.index.values[0]
        i = test_starters.values[0]
    else: 
        s = test_starters.index.values[0]+c
        i = actuals[c-1]
    pcdf.index = np.arange(s+1, len(pcdf)+s+1)
    pcdf.loc[s] = [i]
    pcdf.sort_index(inplace=True)
    pcdf = pcdf.cumsum().round(2)
    pcdf = pcdf.drop(pcdf.index[0])
    c+=1
    pcdf_list.append(pcdf.head(4))

NameError: name 'prediction_chain' is not defined

In [3]:
import ta
import pandas as pd
import numpy as np
import time
from scipy.stats import kurtosis
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots

data = pd.read_csv('NVDA_bar_sample_15secs.csv', index_col=0, header=0)

data = data.loc['2020-09-04 12:35:00':'2020-09-04 14:30:00'].reset_index(drop=False)
# display(data)
data['volume'] = data['volume'].astype('int32')

# del data["average"]
del data["barcount"]

indicator_bb = ta.volatility.BollingerBands(close=data["average"], n=20, ndev=2)
indicator_ao = ta.momentum.AwesomeOscillatorIndicator(high=data["close"], low=data["low"], s= 2, len = 5, fillna=False)
indicator_kama = ta.momentum.kama(data["close"], n=10, pow1=2, pow2=30, fillna=False)
indicator_vpt = ta.volume.VolumePriceTrendIndicator(data['average'], data['volume'], fillna = False)
indicator_macd = ta.trend.MACD(data["close"], n_slow= 24, n_fast= 9, n_sign= 4, fillna= False)
# Add Bollinger Bands features
data['ao'] = indicator_ao.ao()
data['kama'] = indicator_kama
data['vpt'] =  indicator_vpt.volume_price_trend()
data['macd'] = indicator_macd.macd()
data['macd_signal'] = indicator_macd.macd_signal()
data['macd_diff'] = indicator_macd.macd_diff()*2
data['bb_bbm'] = indicator_bb.bollinger_mavg()
data['bb_bbh'] = indicator_bb.bollinger_hband()
data['bb_bbl'] = indicator_bb.bollinger_lband()

# Add Bollinger Band high indicatorabs
# data['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

# # Add Bollinger Band low indicator
# data['bb_bbli'] = indicator_bb.bollinger_lband_indicator()
data = ta.utils.dropna(data)
display(data)

show = data
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)
candlesticks = go.Candlestick(x=show.index,open=show['open'],high=show['high'],low=show['low'],close=show['close'])
bb_bbm = go.Scatter(x=show.index, y=data['bb_bbm'], line=dict(color='black', width=1),name="BB Avg") #,yaxis = "y2"
bb_bbh = go.Scatter(x=show.index, y=data['bb_bbh'], line=dict(color='red', width=1),name="BB High") #,yaxis = "y2"
bb_bbl = go.Scatter(x=show.index, y=data['bb_bbl'], line=dict(color='gray', width=1),name="BB Low") #,yaxis = "y2"
avg = go.Scatter(x=show.index, y=show['average'], line=dict(color='black', width=1),name="Average Price") #,yaxis = "y2"
kama = go.Scatter(x=show.index, y=show['kama'], line=dict(color='orange', width=2),name="Kama")#,yaxis = "y2"
volume = go.Bar(x=show.index, y=show['volume'],yaxis = "y2",name="Volume")
ao = go.Scatter(x=show.index, y=show['ao'], line=dict(color='green', width=2),yaxis = "y3",name="AO")#,yaxis = "y2"
vpt = go.Scatter(x=show.index, y=show['vpt'], line=dict(color='purple', width=2),yaxis = "y4",name="VPT")#,yaxis = "y2"
macd = go.Scatter(x=show.index, y=show['macd'], line=dict(color='blue', width=2),yaxis = "y5",name="MACD")#,yaxis = "y2"
macd_signal = go.Scatter(x=show.index, y=show['macd_signal'], line=dict(color='red', width=2),yaxis = "y5",name="MACD Signal")#,yaxis = "y2"
macd_diff = go.Bar(x=show.index, y=show['macd_diff'],yaxis = "y5",name="MACD Diff")#,yaxis = "y2"
chart_full = [candlesticks,bb_bbm,bb_bbh,bb_bbl,kama,avg,volume,ao,vpt,macd,macd_signal,macd_diff]

layout = go.Layout( yaxis=dict(domain=[.7, 1]),legend=dict(traceorder="reversed"),yaxis2=dict(domain=[.6, .7]),yaxis3=dict(domain=[.5, .6]),yaxis4=dict(domain=[.4, .5]),yaxis5=dict(domain=[.3, .4]))

fig = go.Figure(data=chart_full,layout=layout)
fig.update_layout(xaxis_rangeslider_visible=True,height=1600)
fig.show()

test_data,test_starters,prediction,actuals,prediction_chain, mse, rmse, mape = calcArima(data,"ao",len_test=30,diff =1)

starting_index = test_data.iloc[[0]].index.values[0]
# print(actuals)
# print(starting_index)

# print(test_starters.index.values[0])
pcdf_list = list()
c = 0
for x in prediction_chain:
    pcdf = pd.DataFrame(x)
    pcdf.columns = ['pc']
    if c == 0: 
        s = test_starters.index.values[0]
        i = test_starters.values[0]
    else: 
        s = test_starters.index.values[0]+c
        i = actuals[c-1]
    pcdf.index = np.arange(s+1, len(pcdf)+s+1)
    pcdf.loc[s] = [i]
    pcdf.sort_index(inplace=True)
    pcdf = pcdf.cumsum().round(2)
    pcdf = pcdf.drop(pcdf.index[0])
    c+=1
    pcdf_list.append(pcdf.head(2))





show = test_data
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)
candlesticks = go.Candlestick(x=show.index,open=show['open'],high=show['high'],low=show['low'],close=show['close'])
pred = go.Scatter(x=show.index, y=prediction, line=dict(color='orange', width=2),yaxis = "y2")#,yaxis = "y2"
act = go.Scatter(x=show.index, y=actuals, line=dict(color='black', width=1),yaxis = "y2")#,yaxis = "y2"

# pred = go.Scatter(x=pcdf_list[4].index, y=pcdf_list[4]['pc'], line=dict(color='red', width=2))
chart_full = [candlesticks,pred,act]

layout = go.Layout( yaxis=dict(domain=[.5, 1]),legend=dict(traceorder="reversed"),yaxis2=dict(domain=[0, .5]),yaxis3=dict(domain=[0, .2]),)

fig = go.Figure(data=chart_full,layout=layout)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()


# for x in pcdf_list:
#     pc = go.Scatter(x=x.index, y=x['pc'], line=dict(color='red', width=2),yaxis='y2')
#     chart_full = [candlesticks,act,pred,pc]
#     fig = go.Figure(data=chart_full,layout=layout)
#     fig.update_layout(xaxis_rangeslider_visible=False)
#     fig.show()
#     time.sleep(1)

ModuleNotFoundError: No module named 'ta'

In [ ]:
show = test_data
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)
candlesticks = go.Candlestick(x=show.index,open=show['open'],high=show['high'],low=show['low'],close=show['close'])
pred = go.Scatter(x=show.index, y=prediction, line=dict(color='orange', width=2),yaxis = "y2")#,yaxis = "y2"
act = go.Scatter(x=show.index, y=actuals, line=dict(color='black', width=1),yaxis = "y2")#,yaxis = "y2"

# pred = go.Scatter(x=pcdf_list[4].index, y=pcdf_list[4]['pc'], line=dict(color='red', width=2))
chart_full = [candlesticks,pred,act]

layout = go.Layout( yaxis=dict(domain=[.5, 1]),legend=dict(traceorder="reversed"),yaxis2=dict(domain=[0, .5]),yaxis3=dict(domain=[0, .2]),)

fig = go.Figure(data=chart_full,layout=layout)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()


for x in pcdf_list:
    pc = go.Scatter(x=x.index, y=x['pc'], line=dict(color='red', width=2),yaxis='y2')
#     pred = go.Scatter(x=x.index, y=prediction, line=dict(color='orange', width=2),yaxis='y2')
#     act = go.Scatter(x=show.index, y=actuals, line=dict(color='black', width=1),yaxis = "y2")#,yaxis = "y2"
    chart_full = [candlesticks,act,pred,pc]
    fig = go.Figure(data=chart_full,layout=layout)
    fig.update_layout(xaxis_rangeslider_visible=False)
#     fig.update_traces(overwrite=True)
    fig.show()
    time.sleep(1)

In [ ]:


show = test_data



fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)
candlesticks = go.Candlestick(x=show.index,open=show['open'],high=show['high'],low=show['low'],close=show['close'])
pred = go.Scatter(x=show.index, y=prediction, line=dict(color='orange', width=2),yaxis = "y2")#,yaxis = "y2"
act = go.Scatter(x=show.index, y=actuals, line=dict(color='black', width=1),yaxis = "y2")#,yaxis = "y2"
chart_full = [candlesticks,pred,act]

layout = go.Layout( yaxis=dict(domain=[.9, 1]),legend=dict(traceorder="reversed"),yaxis2=dict(domain=[0, .9]),yaxis3=dict(domain=[0, .9]),)

fig = go.Figure(data=chart_full,layout=layout)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(title='Animation Test',
                  title_x=0.5,
                 height=800, 
                  xaxis_title='Time', 
                  yaxis_title='Test Points',
#                   yaxis_range=(0,99),
#                   xaxis_range=(0,99), #you generate y-values for i =0, ...99, 
                                      #that are assigned, by default, to x-values 0, 1, ..., 99
                  
                  updatemenus=[dict(buttons = [dict(
                                               args = [None, {"frame": {"duration": 400, 
                                                                        "redraw": True},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 200}}],
                                               label = "Play",
                                               method = "animate")],
                                type='buttons',
                                showactive=False,
                                y=1,
                                x=1.12,
                                xanchor='right',
                                yanchor='top')])
                                          
# def makeFrames(pcdf_list):
#     fl = []
#     for x in pcdf_list:
frames= [go.Frame(data=[go.Scatter(x=i.index, y=i['pc'], line=dict(color='red', width=2),yaxis='y2')]) for i in pcdf_list]
fig.update(frames=frames)
